# Data Preprocessing

- akan melakukan preprocessing corpus hasil scraping website halodoc menggunakan stopwords indonesia dan stemming dengan sastrawi
- untuk fine-tuning model indoBERT

## Import and Initializing

In [1]:
# import
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# load stopwords
stop_words = set(stopwords.words('indonesian'))

# load stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()



In [2]:
# load dataset
df = pd.read_csv('scraping/halodoc1-1000.csv')
print(df.shape)

# check dataset
df.head()

(9986, 5)


,url,title,title_length,content,content_count
0,https://www.halodoc.com/ketahui-segala-hal-men...,Ketahui Segala Hal Mengenai COVID-19,38,“Infeksi COVID-19 adalah salah satu penyakit y...,6532
1,https://www.halodoc.com/bagaimana-cara-merawat...,Bagaimana Cara Merawat Karies Gigi?,37,“Ada banyak pilihan perawatan medis dan rumaha...,5993
2,https://www.halodoc.com/cegah-burnout-ini-tand...,"Cegah Burnout, Ini Tanda Pekerja Butuh Piknik...",61,“Burnout merupakan kondisi ketika seseorang m...,4982
3,https://www.halodoc.com/kenalan-dengan-terapi-...,Kenalan dengan Terapi ABA untuk Anak Autis,44,“Salah satu cara untuk mengurangi gejala auti...,4855
4,https://www.halodoc.com/10-herbal-untuk-mereda...,10 Herbal untuk Meredakan Sakit Perut Bagian ...,52,“Ada banyak herbal yang dapat membantu mereda...,3638


In [3]:
# check null values
df.isnull().sum()

url              0
title            0
title_length     0
content          0
content_count    0
dtype: int64

## Data Preprocessing

In [4]:
# check duplicates
print(df.duplicated().sum())
print(f"Ukuran dataset sebelum preprocessing: {df.shape}")

# drop duplicates
df.drop_duplicates(inplace=True)
print(f"Ukuran dataset setelah preprocessing: {df.shape}")

5
Ukuran dataset sebelum preprocessing: (9986, 5)
Ukuran dataset setelah preprocessing: (9981, 5)


### Stopwords Removal, Stemming, etc.

In [6]:
# stopword removal
def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop_words]
    return " ".join(text)

df['title'] = df['title'].apply(remove_stopwords)

# stemming
def stemming(text):
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text)

df['title'] = df['title'].apply(stemming)

# remove punctuation
def remove_punctuation(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text

df['title'] = df['title'].apply(remove_punctuation)

# remove numbers
def remove_numbers(text):
    text = re.sub(r'\d+', '', text)
    return text

df['title'] = df['title'].apply(remove_numbers)


df.head()

,url,title,title_length,content,content_count
0,https://www.halodoc.com/ketahui-segala-hal-men...,tahu covid,38,“Infeksi COVID-19 adalah salah satu penyakit y...,6532
1,https://www.halodoc.com/bagaimana-cara-merawat...,rawat karies gigi,37,“Ada banyak pilihan perawatan medis dan rumaha...,5993
2,https://www.halodoc.com/cegah-burnout-ini-tand...,cegah burnout tanda kerja butuh piknik istirahat,61,“Burnout merupakan kondisi ketika seseorang m...,4982
3,https://www.halodoc.com/kenalan-dengan-terapi-...,kenal terapi aba anak autis,44,“Salah satu cara untuk mengurangi gejala auti...,4855
4,https://www.halodoc.com/10-herbal-untuk-mereda...,herbal reda sakit perut,52,“Ada banyak herbal yang dapat membantu mereda...,3638


In [11]:
# reassign 'title_length' and 'content_length' in df dengan panjang dari 'title' dan 'content' yang baru (character count)
df['title_length'] = df['title'].apply(lambda x: len(x))
df['content_count'] = df['content'].apply(lambda x: len(x))

df.head()

,url,title,title_length,content,content_count
0,https://www.halodoc.com/ketahui-segala-hal-men...,tahu covid,10,“Infeksi COVID-19 adalah salah satu penyakit y...,6532
1,https://www.halodoc.com/bagaimana-cara-merawat...,rawat karies gigi,17,“Ada banyak pilihan perawatan medis dan rumaha...,5993
2,https://www.halodoc.com/cegah-burnout-ini-tand...,cegah burnout tanda kerja butuh piknik istirahat,48,“Burnout merupakan kondisi ketika seseorang m...,4982
3,https://www.halodoc.com/kenalan-dengan-terapi-...,kenal terapi aba anak autis,27,“Salah satu cara untuk mengurangi gejala auti...,4855
4,https://www.halodoc.com/10-herbal-untuk-mereda...,herbal reda sakit perut,24,“Ada banyak herbal yang dapat membantu mereda...,3638


In [12]:
# save to csv
df.to_csv('halodoc1-1000_preprocessed.csv', index=False)